In [ ]:
import os
from IPython.core.display import display, HTML, Markdown, clear_output, Javascript
import ipywidgets as widgets
from IPython.display import IFrame, display

def on_change(change): # si attiva quando cambio scelta dell'elemento da selezionare
    global attach
    if change['type'] == 'change' and change['name'] == 'value':
        #print ("Stai scegliendo %s" % change['new'])
        attach = change['new']
        return attach

def on_button_update(_):
    clear_output()
    #display(Javascript('''var c = IPython.notebook.get_selected_index();
                       #IPython.notebook.execute_cells([c])'''))
    allegato()

def open_attachment(attach_name):
    path_attachments = "allegati"
    path_file=str(path_attachments + "/" + attach_name)
    ext = os.path.splitext(path_file)[-1].lower()
    #print(path_file)
    #print(ext)
    #print(ext.replace('.', ''))
    if(ext=='.pdf'):
        pdf=IFrame(path_file, width=700, height=400)
        display(pdf)
    else:
        #file = open(path_file, "rb")
        display(Markdown("![alt text](" + path_file + ")"))
        delete = widgets.Button(description='Rimuovi allegato')
        delete.on_click(on_button_delete)
        display(delete)

        #image = file.read()
        #w=widgets.Image(
        #    value=image,
        #    format=ext.replace('.', ''),
        #    width=300,
        #    height=400,
        #)
        #display(w)

def on_button_confirm(_):
    global attach
    clear_output()
    print('Allegato: ' + str(attach))
    open_attachment(attach)

def allegato():
    path_attachments = "allegati"
    global attach # allegato attuale scelto
    update = widgets.Button(description='Aggiorna') #bottone visualizzato in ogni caso
    delete = widgets.Button(description='Rimuovi allegato')
    attach_list = os.listdir(path_attachments)
    if 'ck_points' in attach_list:
        attach_list.remove('ck_points')

    if(attach_list): #si attiva se c'è almeno un elemento in lista quindi nella cartella allegati
        confirm = widgets.Button(description='Conferma')
        chosen_attach_wid = widgets.Dropdown(
            options = attach_list,
            value = attach_list[0],
            description = 'Allegati:',
            disabled = False,
        )
        attach = attach_list[0]
        chosen_attach_wid.observe(on_change)
        update.on_click(on_button_update)
        confirm.on_click(on_button_confirm)
        delete.on_click(on_button_delete)
        all_buttons = [update, chosen_attach_wid, confirm, delete]
        display(widgets.HBox(all_buttons))
    else:
        print(f"ATTENZIONE: non sono presenti file nella cartella allegati.") #Ti ricordiamo che la cartella utile per gli allegati da te prodotti per questo esercizio in modo_libero di svolgimento è la cartella @path_ex_folder@/modo_libero/allegati/")
        update.on_click(on_button_update)
        delete.on_click(on_button_delete)
        all_buttons = [update, delete]
        display(widgets.HBox(all_buttons))

def on_button_delete(_):
    #delete_above_cell()
    delete_this_cell()

def delete_above_cell():
    display(Javascript('''var c = IPython.notebook.get_selected_index();
                                IPython.notebook.get_cell(c-1).metadata.editable = true;
                                IPython.notebook.get_cell(c-1).metadata.deletable = true;
                                IPython.notebook.delete_cell([c-1]);'''))
def delete_this_cell():
    display(Javascript('''var c = IPython.notebook.get_selected_index();
                                IPython.notebook.get_cell(c).metadata.editable = true;
                                IPython.notebook.get_cell(c).metadata.deletable = true;
                                IPython.notebook.delete_cell([c]);'''))

def on_button_raw_attach(_):
    add_attach_cell_din('allegato()')

def on_button_md(_):
    add_md_cell_din()

def on_button_code(_):
    add_code_cell_din()

def on_button_raw(_):
    add_raw_cell_din()

def add_raw_cell_din():
    display_id = int(time.time()*1e9) # Hack
    display(Javascript('''var idx = IPython.notebook.get_selected_index();
                       var c = IPython.notebook.insert_cell_at_index("raw", idx);
    c.set_text('');
    var t_index = IPython.notebook.get_cells().indexOf(c);
    IPython.notebook.to_raw(t_index);
    IPython.notebook.get_cell(t_index).render();
    IPython.notebook.get_cell(t_index).metadata.deletable = true;
    IPython.notebook.get_cell(t_index).set_text('');;'''),display_id=display_id) # Hack
    # Necessary hack to avoid self-generation of cells at notebook re-opening
    # See http://tiny.cc/fnf3nz
    display(Javascript(""" """), display_id=display_id, update=True)
    return

def add_attach_cell_din(code = ''):
    display_id = int(time.time()*1e9) # Hack
    display(Javascript('''var c = IPython.notebook.insert_cell_above();
    c.set_text(' ''' + code + ''' ');
    var t_index = IPython.notebook.get_cells().indexOf(c);
    IPython.notebook.to_code(t_index);
    IPython.notebook.get_cell(t_index).render();
    IPython.notebook.execute_cells([t_index]);
    IPython.notebook.get_cell(t_index).metadata.editable = false;'''),display_id=display_id) # Hack
    # Necessary hack to avoid self-generation of cells at notebook re-opening
    # See http://tiny.cc/fnf3nz
    display(Javascript(""" """), display_id=display_id, update=True)
    return

def add_code_cell_din(code = ''):
    display_id = int(time.time()*1e9)
    display(Javascript('''var c = IPython.notebook.insert_cell_above();
    c.set_text(' ''' + code + ''' ');
    var t_index = IPython.notebook.get_cells().indexOf(c);
    IPython.notebook.to_code(t_index);
    IPython.notebook.get_cell(t_index).render();
    IPython.notebook.execute_cells([t_index]);
    IPython.notebook.get_cell(t_index).metadata.deletable = true;
    IPython.notebook.get_cell(t_index).set_text('');'''),display_id=display_id)
    # Necessary hack to avoid self-generation of cells at notebook re-opening
    # See http://tiny.cc/fnf3nz
    display(Javascript(""" """), display_id=display_id, update=True)
    return

def add_md_cell_din():
    display_id = int(time.time()*1e9)
    display(Javascript('''var c = IPython.notebook.insert_cell_above();
    c.set_text(' ');
    var t_index = IPython.notebook.get_cells().indexOf(c);
    IPython.notebook.to_markdown(t_index);
    IPython.notebook.get_cell(t_index).render();
    IPython.notebook.get_cell(t_index).metadata.deletable = true;
    IPython.notebook.get_cell(t_index).set_text('');'''),display_id=display_id) # Hack
    # Necessary hack to avoid self-generation of cells at notebook re-opening
    # See http://tiny.cc/fnf3nz
    display(Javascript(""" """), display_id=display_id, update=True)
    return

def add_cell(code='', position='below', celltype='markdown', is_execute = False):
    """Create a cell in the IPython Notebook.
    code: unicode, Code to fill the new cell with.
    celltype: unicode, Type of cells "code" or "markdown".
    position: unicode, Where to put the cell "below" or "at_bottom"
    is_execute: boolean, To decide if the cell is executed after creation
    """

    # Create a unique id based on epoch time
    display_id = int(time.time()*1e9)

    if is_execute:
        display(Javascript("""
        var basis = IPython.notebook.insert_cell_{0}("{1}");
        basis.set_text(atob("{2}"));
        basis.execute();
        """.format(position, celltype, " ")),display_id=display_id)

    else:
        display(Javascript("""
        var basis = IPython.notebook.insert_cell_{0}("{1}");
        basis.set_text(atob("{2}"));
        """.format(position, celltype, " ")),display_id=display_id)


    # Necessary hack to avoid self-generation of cells at notebook re-opening
    # See http://tiny.cc/fnf3nz
    display(Javascript(""" """), display_id=display_id, update=True)

def code_button_delete():
    button_delete = widgets.Button(description="Rimuovi l'allegato", tooltip="Seleziona la cella e clicca su Elimina")
    button_delete.on_click(on_button_delete)
    #print("Vuoi eliminare la cella selezionata?")
    display(button_delete)

def loader_main():
    button_raw_attach = widgets.Button(description="Rispondi (Allegato)", tooltip="Collega un file dalla cartella attachments")
    button_md = widgets.Button(description="Rispondi (Markdown)", abstooltip="Aggiungi una cella per scrivere del testo in Markdown")
    button_code = widgets.Button(description="Rispondi (Code)", tooltip="Aggiungi una cella per scrivere del codice in Python")
    button_raw = widgets.Button(description="Rispondi (Raw)", tooltip="Aggiungi una cella per scrivere del testo libero (SCONSIGLIATA: righe troppo lunghe potrebbero fuoriuscire nella rendition, usala solo se non riesci a controllare il Markdown)")
    button_raw_attach.on_click(on_button_raw_attach)
    button_md.on_click(on_button_md)
    button_code.on_click(on_button_code)
    button_raw.on_click(on_button_raw)

    all_buttons = [button_code, button_md, button_raw, button_raw_attach]
    display(widgets.HBox(all_buttons))


# ESERCIZIO: Massima sottosequenza comune #


In [ ]:
import os
from IPython.core.display import display, HTML, Markdown, clear_output, Javascript
import ipywidgets as widgets
from IPython.display import IFrame, display
import numpy as np


In [ ]:
    n_tasks = 1;
    arr_point = [-1] * n_tasks;

In [ ]:
    s = "CCCCCAAACTATATATA"
    t = "GTCACGTACGTACAAAA"
    

In [ ]:
    def is_sub(sub,string):
        match=0
        i=0
        j=0
        sol=True
        while match<len(sub) and i<len(string):
            if string[i]==sub[j]:
                match+=1
                if j<len(sub)-1:
                    j+=1
            i+=1
        if match!=len(sub):
            sol=False
        return sol

    def verifLCS(string1, string2, answer, pt_green, pt_red, index_pt, start=False, end=False):
        if answer=="":
            return evaluation_format("No", 0, pt_red, index_pt)+f"La sottosequenza fornita è vuota."
        if start != False and answer[0]!=start:
            return evaluation_format("No", 0, pt_red, index_pt)+f"La sottosequenza fornita non inizia con __{start}__."
        if end != False and answer[len(answer)-1]!=end:
            return evaluation_format("No", 0, pt_red, index_pt)+f"La sottosequenza fornita non termina con __{end}__."
        s = ' ' + string1 
        t = ' ' + string2
        n = len(s)
        m = len(t)
        L = np.zeros((n, m)) 
        for i in range(1,n):
            for j in range(1,m):
                if s[i] == t[j]: 
                    L[i][j] = L[i-1][j-1] + 1
                else: 
                    L[i][j] = max(L[i-1][j], L[i][j-1]) 
        correct_len=np.max(L)
        if (is_sub(answer,string1)) and (is_sub(answer,string2)):
            return evaluation_format("Si", pt_green, pt_red, index_pt)+"La sottosequenza fornita è ammissibile."
        else:
            return evaluation_format("No", 0, pt_red, index_pt)+"La sottosequenza non è ammissibile, rincontrolla che sia una sottosequenza di entrambe."
    
    def evaluation_format(answ, pt_green,pt_red, index_pt):
        pt_blue=0
        if pt_green!=0:
            pt_blue=pt_red-pt_green
            pt_red=0
        arr_point[index_pt]=pt_green
        file = open("points.txt", "w")
        file.write(str(arr_point))
        file.close()
        return f"{answ}. Totalizzeresti <span style='color:green'>[{pt_green} safe pt]</span>,                                         <span style='color:blue'>[{pt_blue} possible pt]</span>,                                         <span style='color:red'>[{pt_red} out of reach pt]</span>.<br>"

    

Si considerino le seguenti sequenze di caratteri:<br/><br/> $s$ = CCCCCAAACTATATATA<br/>  $t$ = GTCACGTACGTACAAAA

__Richiesta 0 [10 punti]__: Fornire una massima sottosequenza comune tra le <b> due stringhe</b>:<br/>$s$ = CCCCCAAACTATATATA<br/>$t$ = GTCACGTACGTACAAAA.

In [ ]:
#Inserisci la risposta (una stringa, per esempio "TATATA")
answer0=""

In [ ]:
display(Markdown(verifLCS(s, t ,answer0,pt_green=1, pt_red=10, index_pt=0)))

In [ ]:
import os
import time
from urllib.request import urlopen
import nbformat
from traitlets.config import Config
from nbconvert import HTMLExporter
from nbconvert import RSTExporter, NotebookExporter
from IPython.display import Image, HTML, Javascript
from datetime import datetime

import shutil
last ='preview_last/' #path of last submission
old = 'preview_old/' #path of prevous submissions

#hide alert buttons
def hide_w(w):
    for e in w:
        e.layout.visibility = 'hidden'

#show alert buttons
def show_w(w):
    for e in w:
        e.layout.visibility = 'visible'

def are_you_sure():
    show_w(alert_buttons)

label = widgets.Label(value="Un tuo elaborato era già stato precedentemente prodotto. Vuoi rimpiazzarlo con il presente?")
yes = widgets.Button(description="Si", tooltip="Produce il nuovo elaborato. Esso diviene la versione attuale prendendo il posto del precedente")
no = widgets.Button(description="No")

alert_buttons = [label,yes,no]
hide_w(alert_buttons)

#move all files from last to old and save the new preview
def yes_clicked(b):
    files = os.listdir(last)
    for f in files:
        if f == '.ipynb_checkpoints':
            continue
        if not os.path.isdir('./preview_old'):
            os.mkdir(old)
        shutil.move(last+f, old)
    display(Javascript('IPython.notebook.save_checkpoint();'))
    now = datetime.now() # current date and time
    date_time = now.strftime("%Y-%m-%d-%H-%M-%S")
    hname = 'Esercizio_01.ipynb'[:-6] + '_' + date_time
    ! jupyter nbconvert Esercizio_01.ipynb --to html_embed --output=$hname --output-dir=./preview_last/ --TagRemovePreprocessor.remove_cell_tags='["noexport"]'
    display(Javascript('window.open("./preview_last/' + hname + '.html")'))
    hide_w(alert_buttons)
def no_clicked(b):
    hide_w(alert_buttons)


def generate_preview_HTML(_):
    if not os.path.isdir('./preview_last'):
        os.mkdir(last)
    directory= os.listdir(last)
    if len(directory) <1:
        display(Javascript('IPython.notebook.save_checkpoint();'))
        now = datetime.now() # current date and time
        date_time = now.strftime("%Y-%m-%d-%H-%M-%S")
        hname = 'Esercizio_01.ipynb'[:-6] + '_' + date_time
        ! jupyter nbconvert Esercizio_01.ipynb --to html_embed --output=$hname --output-dir=./preview_last/ --TagRemovePreprocessor.remove_cell_tags='["noexport"]'
        display(Javascript('window.open("./preview_last/' + hname + '.html")'))
    else:
        are_you_sure()


button = widgets.Button(description="Salva & Esporta", tooltip="Esporta il foglio Jupyter in HTML nella cartella preview")
output = widgets.Output()
button.on_click(generate_preview_HTML)
yes.on_click(yes_clicked)
no.on_click(no_clicked)
h_box1 = widgets.HBox([button])
h_box2 = widgets.HBox([label,yes,no])
display(widgets.VBox([h_box1,h_box2]))
